In [1]:
# rm(list=ls())
require(foreign)
require(readstata13)
require(data.table)
require(MASS)
require(ggplot2)
# require(foreach)
require(gridExtra)
require(stringr)
require(matrixStats)
require(parallel)
require(abind)
require(rlist)


parallel::detectCores()
## Resizing notebook plot space
options(repr.plot.width=16, repr.plot.height=9)

Loading required package: foreign
Loading required package: readstata13
Loading required package: data.table
Loading required package: MASS
Loading required package: ggplot2
Loading required package: gridExtra
Loading required package: stringr
Loading required package: matrixStats
matrixStats v0.51.0 (2016-10-08) successfully loaded. See ?matrixStats for help.
Loading required package: parallel
Loading required package: abind
Loading required package: rlist


[1] 40

In [2]:
### Bring in location metadata
locs <- data.table(read.dta13("/home/j/Project/IRH/DAH/RESEARCH/INTEGRATED DATABASES/DATA/FGH_2016/region_data/location_metadata.dta"))
countries <- unique(locs[, location_id])

## Bring in LDI, TFR and Education

In [9]:
ldi <- data.table(fread('/share/forecasting/data/LDIpc/national_LDIpc_prepped_20161207.csv'))
ldi <- ldi[location_id %in% countries, ]
head(ldi)

Read 12090 rows and 1008 (of 1008) columns from 0.112 GB file in 00:00:03


location_id,ihme_loc_id,year_id,age_group_id,sex_id,LDIpc_1,LDIpc_2,LDIpc_3,LDIpc_4,LDIpc_5,⋯,LDIpc_994,LDIpc_995,LDIpc_996,LDIpc_997,LDIpc_998,LDIpc_999,LDIpc_1000,rt_mean,rt_upper,rt_lower
6,CHN,1979,22,3,1015.737,1015.737,1015.737,1015.737,1015.737,⋯,1015.737,1015.737,1015.737,1015.737,1015.737,1015.737,1015.737,1015.737,1015.737,1015.737
6,CHN,1980,22,3,1028.146,1028.146,1028.146,1028.146,1028.146,⋯,1028.146,1028.146,1028.146,1028.146,1028.146,1028.146,1028.146,1028.146,1028.146,1028.146
6,CHN,1981,22,3,1044.316,1044.316,1044.316,1044.316,1044.316,⋯,1044.316,1044.316,1044.316,1044.316,1044.316,1044.316,1044.316,1044.316,1044.316,1044.316
6,CHN,1982,22,3,1075.913,1075.913,1075.913,1075.913,1075.913,⋯,1075.913,1075.913,1075.913,1075.913,1075.913,1075.913,1075.913,1075.913,1075.913,1075.913
6,CHN,1983,22,3,1114.356,1114.356,1114.356,1114.356,1114.356,⋯,1114.356,1114.356,1114.356,1114.356,1114.356,1114.356,1114.356,1114.356,1114.356,1114.356
6,CHN,1984,22,3,1171.506,1171.506,1171.505,1171.505,1171.505,⋯,1171.505,1171.505,1171.506,1171.505,1171.505,1171.505,1171.505,1171.506,1171.506,1171.505


In [10]:
tfr <- data.table(fread("/ihme/forecasting/data/tfr/results/tfr_whole.csv"))
tfr <- tfr[, mean := rowMeans(.SD), .SDcols = paste0("draw_",c(0:999))]
tfr <- tfr[location_id %in% countries, ]
head(tfr)

Read 16748 rows and 1004 (of 1004) columns from 0.113 GB file in 00:00:04


age_group_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,⋯,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,sex_id,year_id,mean
22,6.8066,6.8066,6.8066,6.8066,6.8066,6.8066,6.8066,6.8066,6.8066,⋯,6.8066,6.8066,6.8066,6.8066,6.8066,6.8066,175,3,1953,6.8066
22,6.8178,6.8178,6.8178,6.8178,6.8178,6.8178,6.8178,6.8178,6.8178,⋯,6.8178,6.8178,6.8178,6.8178,6.8178,6.8178,175,3,1954,6.8178
22,6.8290,6.8290,6.8290,6.8290,6.8290,6.8290,6.8290,6.8290,6.8290,⋯,6.8290,6.8290,6.8290,6.8290,6.8290,6.8290,175,3,1955,6.8290
22,6.8402,6.8402,6.8402,6.8402,6.8402,6.8402,6.8402,6.8402,6.8402,⋯,6.8402,6.8402,6.8402,6.8402,6.8402,6.8402,175,3,1956,6.8402
22,6.8514,6.8514,6.8514,6.8514,6.8514,6.8514,6.8514,6.8514,6.8514,⋯,6.8514,6.8514,6.8514,6.8514,6.8514,6.8514,175,3,1957,6.8514
22,6.8784,6.8784,6.8784,6.8784,6.8784,6.8784,6.8784,6.8784,6.8784,⋯,6.8784,6.8784,6.8784,6.8784,6.8784,6.8784,175,3,1958,6.8784


In [11]:
edu <- data.table(fread("/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/country_year_A_S_T_collapsed.csv"))
edu <- edu[location_id %in% countries, ]
head(edu)

Read 17321 rows and 1005 (of 1005) columns from 0.274 GB file in 00:00:05


location_id,year_id,draw0,draw1,draw2,draw3,draw4,draw5,draw6,draw7,⋯,draw993,draw994,draw995,draw996,draw997,draw998,draw999,mean,lower,upper
10,1950,1.336208,1.678651,1.666961,1.321678,1.726451,2.100176,1.644592,1.935168,⋯,1.805843,1.324314,1.176254,1.550790,2.723640,1.806279,2.049624,1.645919,1.068804,2.630491
10,1951,1.363233,1.692118,1.694677,1.350223,1.766093,2.145282,1.688327,1.959271,⋯,1.823794,1.364587,1.208113,1.570845,2.754895,1.836829,2.107365,1.675652,1.087175,2.682010
10,1952,1.391230,1.705612,1.723597,1.380729,1.806876,2.189708,1.732409,1.983210,⋯,1.841873,1.406279,1.241448,1.591174,2.785191,1.867273,2.165253,1.706197,1.111213,2.737125
10,1953,1.420162,1.719183,1.753810,1.413282,1.848761,2.233337,1.776700,2.007246,⋯,1.860134,1.449185,1.276323,1.611863,2.814625,1.897611,2.223214,1.737582,1.135241,2.791019
10,1954,1.449966,1.732866,1.785342,1.447930,1.891650,2.276010,1.821005,2.031560,⋯,1.878600,1.493041,1.312772,1.633004,2.843222,1.927806,2.281070,1.769783,1.160084,2.835183
10,1955,1.480594,1.746735,1.818158,1.484754,1.935420,2.317627,1.865190,2.056326,⋯,1.897400,1.537597,1.350822,1.654735,2.871037,1.957871,2.338589,1.802792,1.190560,2.875104


### The first step is to compute the correlation of in-sample data. Let's subset those out first

In [12]:
ldi_insample <- ldi[year_id<=2015 , .SD, .SDcols = c("location_id", "year_id", "rt_mean")]
colnames(ldi_insample) <- c("location_id", "year_id", "mean.ldi")
edu_insample <- edu[year_id<=2015 & year_id>=1979 , .SD, .SDcols = c("location_id", "year_id", "mean")]
colnames(edu_insample) <- c("location_id", "year_id", "mean.edu")
tfr_insample <- tfr[year_id<=2015 & year_id>=1979 , .SD, .SDcols = c("location_id", "year_id", "mean")]
colnames(tfr_insample) <- c("location_id", "year_id", "mean.tfr")

### Merge and compute correlation

In [13]:
insample_data_merged <- merge(x=ldi_insample, y=edu_insample, by=c("location_id", "year_id" ))
insample_data_merged <- merge(x=insample_data_merged, y=tfr_insample, by=c("location_id", "year_id" ))
head(insample_data_merged)

location_id,year_id,mean.ldi,mean.edu,mean.tfr
6,1979,1015.737,5.776690,2.86566
6,1980,1028.146,5.893375,2.76690
6,1981,1044.316,6.013282,2.66814
6,1982,1075.913,6.133414,2.56938
6,1983,1114.356,6.252339,2.54300
6,1984,1171.506,6.368205,2.58900


### Compute the correlation

In [14]:
print(corr_mat<-cor(insample_data_merged[, .( mean.edu, mean.ldi,mean.tfr)], method = "spearman"))

           mean.edu   mean.ldi   mean.tfr
mean.edu  1.0000000  0.7261548 -0.8285551
mean.ldi  0.7261548  1.0000000 -0.7390126
mean.tfr -0.8285551 -0.7390126  1.0000000


### Neal's univariate copula function

In [15]:
draw2Dcopula <- function(X, cor_mat, df_return = F){
  L <- dim(X)[2]
  D <- dim(X)[3]
  Xsum <- apply(X, c(2, 3), sum)
  mvdat <- mvrnorm(n=D, mu=0 * 1:L, Sigma=cor_mat, empirical=TRUE)
  ranks <- apply(mvdat, 2, rank, ties.method="first")
  sortedXsim <- apply(Xsum, 1, function(x) sort(x, index.return=TRUE)$ix)
  sortedX <- X
  for(i in 1:L){
    sortedX[,i,] <- X[,i,sortedXsim[,i]]
  }
  Xcorr <- sortedX
  for(i in 1:L){
    Xcorr[,i,] <- sortedX[,i,ranks[,i]]
  }
  if (df_return==T) {
    return(data.frame(reshape2::melt(Xcorr)))
    }
      else {
          Xcorr
      }
}

### Merge the draws of LDI and Education, reshape super long and convert to arrays

In [16]:
ldi_fc <- ldi[, .SD, .SDcols = c("location_id", "year_id", paste0("LDIpc_", c(1:1000)))]
ldi_fc <- ldi_fc[, variable := "LDIpc"]
ldi_fc <- setcolorder(ldi_fc, c("location_id", "year_id", "variable", paste0("LDIpc_", c(1:1000))) )
colnames(ldi_fc) <-  c("location_id", "year_id", "variable", paste0("draw_", c(1:1000)))
ldi_fc <- melt(ldi_fc, id.vars=c("location_id","year_id", "variable"), variable.name = "draw_num", value.name = "data")
head(ldi_fc)

location_id,year_id,variable,draw_num,data
6,1979,LDIpc,draw_1,1015.737
6,1980,LDIpc,draw_1,1028.146
6,1981,LDIpc,draw_1,1044.316
6,1982,LDIpc,draw_1,1075.913
6,1983,LDIpc,draw_1,1114.356
6,1984,LDIpc,draw_1,1171.506


In [17]:
edu_fc <- edu[year_id>=1979, .SD, .SDcols = c("location_id", "year_id", paste0("draw", c(0:999)))]
edu_fc <- edu_fc[, variable := "EDU"]
edu_fc <- setcolorder(edu_fc, c("location_id", "year_id", "variable", paste0("draw", c(0:999))) )
colnames(edu_fc) <-  c("location_id", "year_id", "variable", paste0("draw_", c(1:1000)))
edu_fc <- melt(edu_fc, id.vars=c("location_id","year_id", "variable"), variable.name = "draw_num", value.name = "data")
head(edu_fc)

location_id,year_id,variable,draw_num,data
10,1979,EDU,draw_1,2.534496
10,1980,EDU,draw_1,2.588469
10,1981,EDU,draw_1,2.655124
10,1982,EDU,draw_1,2.725430
10,1983,EDU,draw_1,2.798605
10,1984,EDU,draw_1,2.873696


In [18]:
tfr_fc <- tfr[year_id>=1979, .SD, .SDcols = c("location_id", "year_id", paste0("draw_", c(0:999)))]
tfr_fc <- tfr_fc[, variable := "TFR"]
tfr_fc <- setcolorder(tfr_fc, c("location_id", "year_id", "variable", paste0("draw_", c(0:999))) )
colnames(tfr_fc) <-  c("location_id", "year_id", "variable", paste0("draw_", c(1:1000)))
tfr_fc <- melt(tfr_fc, id.vars=c("location_id","year_id", "variable"), variable.name = "draw_num", value.name = "data")
head(tfr_fc)

location_id,year_id,variable,draw_num,data
175,1979,TFR,draw_1,7.4616
175,1980,TFR,draw_1,7.4520
175,1981,TFR,draw_1,7.4424
175,1982,TFR,draw_1,7.4328
175,1983,TFR,draw_1,7.4444
175,1984,TFR,draw_1,7.4772


In [19]:
all_data_binded <- rbind(ldi_fc, edu_fc, tfr_fc)

system.time(myarray2<- lapply(countries, 
                                function(x) reshape2::acast(all_data_binded[location_id == x,], 
                                                      location_id ~  year_id ~ variable ~ draw_num, 
                                                      value.var = "data")))
myarray2 <- abind(myarray2, along=1)
str(myarray2)

   user  system elapsed 
 36.174   0.461  36.619 

 num [1:188, 1:62, 1:3, 1:1000] 2.07 1.95 8.2 9.75 10.65 ...
 - attr(*, "dimnames")=List of 4
  ..$ : chr [1:188] "160" "168" "43" "74" ...
  ..$ : chr [1:62] "1979" "1980" "1981" "1982" ...
  ..$ : chr [1:3] "EDU" "LDIpc" "TFR"
  ..$ : chr [1:1000] "draw_1" "draw_2" "draw_3" "draw_4" ...


### Copula'in time

In [21]:
system.time(Xcorr_array <- lapply(countries , 
                                     function(x) cbind(x, draw2Dcopula(myarray2[paste0(x),,,], corr_mat , df_return = T)) ))
                                        
test2 <- list.stack(Xcorr_array, data.table=T)
colnames(test2) <- c("location_id", "year_id", "variable", "draw_num", "data")
tail(test2)                  

   user  system elapsed 
 16.475   0.003  16.476 

location_id,year_id,variable,draw_num,data
198,2035,TFR,draw_1000,2.75927
198,2036,TFR,draw_1000,2.66840
198,2037,TFR,draw_1000,2.58057
198,2038,TFR,draw_1000,2.49567
198,2039,TFR,draw_1000,2.48211
198,2040,TFR,draw_1000,2.46863


In [22]:
### Stack into a DT
test3 <- dcast(test2, location_id + year_id + variable ~ draw_num, value.var = c("data"))
head(test3)

location_id,year_id,variable,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,⋯,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,draw_1000
6,1979,EDU,5.281507,5.691998,5.981159,5.228214,5.402339,5.933942,5.480035,⋯,6.622086,5.974635,5.863802,5.994738,5.596356,5.199357,6.273481,5.436187,5.878156,5.481549
6,1979,LDIpc,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,⋯,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700
6,1979,TFR,2.865660,2.865660,2.865660,2.865660,2.865660,2.865660,2.865660,⋯,2.865660,2.865660,2.865660,2.865660,2.865660,2.865660,2.865660,2.865660,2.865660,2.865660
6,1980,EDU,5.409671,5.800328,6.055033,5.357602,5.535630,6.035072,5.610173,⋯,6.692808,6.101372,6.004011,6.069328,5.712001,5.352301,6.361074,5.587738,5.978882,5.615926
6,1980,LDIpc,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,⋯,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400
6,1980,TFR,2.766900,2.766900,2.766900,2.766900,2.766900,2.766900,2.766900,⋯,2.766900,2.766900,2.766900,2.766900,2.766900,2.766900,2.766900,2.766900,2.766900,2.766900


In [24]:
LDI_sorted <- test3[variable=="LDIpc", .SD, .SDcols = c("location_id", "year_id", paste0("draw_", c(1:1000)))]
colnames(LDI_sorted) <- c("location_id", "year_id", paste0("LDIpc_", c(1:1000)))
EDU_sorted <- test3[variable=="EDU", .SD, .SDcols = c("location_id", "year_id", paste0("draw_", c(1:1000)))]
colnames(EDU_sorted) <- c("location_id", "year_id", paste0("draw", c(0:999)))
TFR_sorted <- test3[variable=="TFR", .SD, .SDcols = c("location_id", "year_id", paste0("draw_", c(1:1000)))]
colnames(TFR_sorted) <- c("location_id", "year_id", paste0("draw", c(0:999)))

In [25]:
## Save out
fwrite(LDI_sorted, "/ihme/forecasting/data/covariates/ldi_per_capita/national_LDIpc_corrd_with_EDU_20170211.csv")
fwrite(EDU_sorted, "/ihme/forecasting/data/covariates/education/national_EDU_corrd_with_LDIpc_20170211.csv")
fwrite(TFR_sorted, "/ihme/forecasting/data/covariates/TFR/national_TFR_corrd_with_LDIpc_EDU_20170211.csv")